In [ ]:
# 03_Case_Retrieval_BERT_TF-IDF.ipynb

import pandas as pd
import numpy as np
import torch
from transformers import AutoTokenizer, AutoModel
from sklearn.metrics.pairwise import cosine_similarity
import os
import json
from google.colab import drive

In [ ]:
# Mount Google Drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# --- Konfigurasi Path (sesuaikan dengan struktur proyek Anda) ---
BASE_DRIVE_PATH = "/content/drive/MyDrive/Semester 6/PK/UAS" # Sesuaikan jika berbeda
PATH_PROCESSED_DATA = os.path.join(BASE_DRIVE_PATH, "data/processed")
PATH_EVAL_DATA = os.path.join(BASE_DRIVE_PATH, "data/eval")
PATH_MODELS_CACHE = os.path.join(BASE_DRIVE_PATH, "models_cache") # Opsional, untuk cache model Hugging Face

# Membuat direktori jika belum ada
os.makedirs(PATH_EVAL_DATA, exist_ok=True)
os.makedirs(PATH_MODELS_CACHE, exist_ok=True) # Opsional

# File input dari Tahap 2
CASES_REPRESENTED_CSV = os.path.join(PATH_PROCESSED_DATA, "cases_represented.csv") # Atau .json jika Anda simpan sbg JSON

# File output untuk tahap ini
QUERIES_JSON_FILE = os.path.join(PATH_EVAL_DATA, "queries.json")
CASE_EMBEDDINGS_FILE = os.path.join(PATH_PROCESSED_DATA, "case_embeddings_bert.npy") # Untuk menyimpan embeddings
CASE_IDS_FILE = os.path.join(PATH_PROCESSED_DATA, "case_ids_bert.json") # Untuk menyimpan urutan case_id sesuai embeddings


In [ ]:
# --- 1. Muat Data Kasus dari Tahap 2 ---
print(f"Memuat data kasus dari: {CASES_REPRESENTED_CSV}")
try:
    df_cases = pd.read_csv(CASES_REPRESENTED_CSV)
except FileNotFoundError:
    print(f"Error: File {CASES_REPRESENTED_CSV} tidak ditemukan. Pastikan Tahap 2 sudah dijalankan.")
    exit()

if 'text_full' not in df_cases.columns or 'case_id' not in df_cases.columns:
    print("Error: Kolom 'text_full' atau 'case_id' tidak ditemukan dalam CSV. Periksa output Tahap 2.")
    exit()

# Hilangkan baris dengan teks kosong jika ada, karena tidak bisa di-embed
df_cases.dropna(subset=['text_full'], inplace=True)
df_cases = df_cases[df_cases['text_full'].str.strip() != ""]

if df_cases.empty:
    print("Tidak ada kasus dengan teks yang valid untuk diproses. Hentikan.")
    exit()

print(f"Data kasus berhasil dimuat. Jumlah kasus yang akan diproses: {len(df_cases)}")
print(df_cases.head())

Memuat data kasus dari: /content/drive/MyDrive/Semester 6/PK/UAS/data/processed/cases_represented.csv
Data kasus berhasil dimuat. Jumlah kasus yang akan diproses: 18
   case_id             no_perkara     tanggal  \
0        1   28/Pdt.G/2020/PN Sdw         NaN   
1        2  229/Pdt.G/2023/PN Nga         NaN   
2        3   40/Pdt.G/2019/PN Ckr  2019-04-25   
3        4    9/Pdt.G/2025/PA.Sdk         NaN   
4        7   27/Pdt.G/2023/PN Nga         NaN   

                            jenis_perkara  \
0        Perdata Perdata Agama Perceraian   
1        Perdata Perdata Agama Perceraian   
2        Perdata Perdata Agama Perceraian   
3  Perdata Agama Perdata Agama Perceraian   
4        Perdata Perdata Agama Perceraian   

                                   pihak  \
0  Pihak tidak teridentifikasi dari teks   
1  Pihak tidak teridentifikasi dari teks   
2  Penggugat/Pemohon: MERY YANTI L. GAOL   
3             Penggugat/Pemohon: PEMOHON   
4  Pihak tidak teridentifikasi dari teks   

   

# BERT

In [ ]:
# --- 2. Setup Model IndoBERT ---
# Tentukan device (GPU jika tersedia, jika tidak CPU)
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Menggunakan device: {DEVICE}")

# Nama model IndoBERT (sesuai rekomendasi PDF atau pilih yang lain jika perlu) [cite: 198]
MODEL_NAME = 'indobenchmark/indobert-base-p1' # [cite: 198]
print(f"Memuat tokenizer dan model untuk: {MODEL_NAME}...")

try:
    tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME, cache_dir=PATH_MODELS_CACHE)
    model = AutoModel.from_pretrained(MODEL_NAME, cache_dir=PATH_MODELS_CACHE)
    model.to(DEVICE) # Pindahkan model ke device (GPU/CPU)
    model.eval() # Set model ke mode evaluasi
    print("Tokenizer dan model berhasil dimuat.")
except Exception as e:
    print(f"Error saat memuat model atau tokenizer: {e}")
    print("Pastikan Anda memiliki koneksi internet untuk mengunduh model jika belum ada di cache.")
    print("Jika menggunakan Colab, pastikan runtime memiliki akses internet.")
    exit()

Menggunakan device: cuda
Memuat tokenizer dan model untuk: indobenchmark/indobert-base-p1...


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Tokenizer dan model berhasil dimuat.


In [ ]:
# --- 3. Fungsi untuk Menghasilkan Embeddings ---
def get_bert_embedding(text, model, tokenizer, device, max_length=512):
    """
    Menghasilkan embedding untuk teks menggunakan model BERT (IndoBERT).
    Menggunakan strategi Mean Pooling pada last hidden states.
    """
    if not isinstance(text, str) or text.strip() == "":
        # Kembalikan zero vector atau handle error jika teks tidak valid
        # Untuk IndoBERT base, hidden size biasanya 768
        return torch.zeros(model.config.hidden_size).cpu().numpy()

    encoded_input = tokenizer(
        text,
        padding='max_length',    # Pad to max_length
        truncation=True,         # Truncate to max_length
        max_length=max_length,
        return_attention_mask=True,
        return_tensors='pt'      # Return PyTorch tensors
    )
    encoded_input = {k: v.to(device) for k, v in encoded_input.items()}

    with torch.no_grad(): # Tidak perlu menghitung gradien saat inferensi
        outputs = model(**encoded_input)
        last_hidden_states = outputs.last_hidden_state

    # Mean Pooling: rata-rata token embeddings berdasarkan attention mask
    attention_mask = encoded_input['attention_mask']
    mask_expanded = attention_mask.unsqueeze(-1).expand(last_hidden_states.size()).float()
    sum_embeddings = torch.sum(last_hidden_states * mask_expanded, 1)
    sum_mask = torch.clamp(mask_expanded.sum(1), min=1e-9) # Hindari pembagian dengan nol
    mean_pooled = sum_embeddings / sum_mask

    return mean_pooled.cpu().numpy().flatten() # Kembali ke CPU, ubah ke NumPy array 1D

In [ ]:
# --- 4. Generate dan Simpan Embeddings untuk Case Base ---
# Cek apakah embeddings sudah ada untuk menghindari komputasi ulang
if os.path.exists(CASE_EMBEDDINGS_FILE) and os.path.exists(CASE_IDS_FILE):
    print(f"Memuat embeddings yang sudah ada dari {CASE_EMBEDDINGS_FILE}...")
    case_embeddings = np.load(CASE_EMBEDDINGS_FILE)
    with open(CASE_IDS_FILE, 'r') as f:
        case_ids_for_embeddings = json.load(f)
    print(f"Embeddings dan case_ids berhasil dimuat. Jumlah: {len(case_ids_for_embeddings)}")

    # Pastikan jumlah case_ids cocok dengan jumlah baris embeddings
    if len(case_ids_for_embeddings) != case_embeddings.shape[0]:
        print("Peringatan: Jumlah case_ids tidak cocok dengan jumlah embeddings! Akan mengenerate ulang.")
        generate_new_embeddings = True
    else:
        generate_new_embeddings = False
else:
    generate_new_embeddings = True

if generate_new_embeddings:
    print("Menghasilkan embeddings untuk case base (mungkin butuh waktu)...")
    # Gunakan kolom 'text_full' dari df_cases
    # Pastikan df_cases memiliki case_id yang unik dan teks yang valid

    embeddings_list = []
    case_ids_for_embeddings = []

    for index, row in df_cases.iterrows():
        case_id = row['case_id']
        text_to_embed = str(row['text_full']) # Pastikan berupa string

        if not text_to_embed.strip():
            print(f"Peringatan: Teks kosong untuk case_id {case_id}. Dilewati.")
            continue

        print(f"  Processing case_id: {case_id}...")
        embedding = get_bert_embedding(text_to_embed, model, tokenizer, DEVICE)
        embeddings_list.append(embedding)
        case_ids_for_embeddings.append(case_id)

    if not embeddings_list:
        print("Error: Tidak ada embeddings yang berhasil digenerate. Periksa teks input.")
        exit()

    case_embeddings = np.array(embeddings_list)

    print(f"Shape dari case_embeddings: {case_embeddings.shape}")

    # Simpan embeddings dan urutan case_id
    try:
        np.save(CASE_EMBEDDINGS_FILE, case_embeddings)
        print(f"Case embeddings berhasil disimpan ke: {CASE_EMBEDDINGS_FILE}")
        with open(CASE_IDS_FILE, 'w') as f:
            json.dump(case_ids_for_embeddings, f)
        print(f"Urutan Case IDs untuk embeddings disimpan ke: {CASE_IDS_FILE}")
    except Exception as e:
        print(f"Error saat menyimpan embeddings atau case_ids: {e}")

Memuat embeddings yang sudah ada dari /content/drive/MyDrive/Semester 6/PK/UAS/data/processed/case_embeddings_bert.npy...
Embeddings dan case_ids berhasil dimuat. Jumlah: 18


In [ ]:
# --- 5. Implementasi Fungsi Retrieval ---
def retrieve_cases_bert(query_text, k=5):
    """
    Menerima query teks, menghasilkan embeddingnya, menghitung kemiripan
    dengan case base, dan mengembalikan top-k case_id yang paling mirip.
    """
    if not query_text.strip():
        print("Query teks kosong.")
        return []

    print(f"\nMelakukan retrieval untuk query: \"{query_text[:100]}...\"")

    # 1. Preprocessing query (jika ada, tapi biasanya tokenizer BERT menangani)
    # query_processed = preprocess_text_for_bert(query_text) # Jika ada fungsi khusus

    # 2. Hasilkan embedding untuk query
    print("  Menghasilkan embedding untuk query...")
    query_embedding = get_bert_embedding(query_text, model, tokenizer, DEVICE)
    query_embedding = query_embedding.reshape(1, -1) # Reshape untuk cosine_similarity

    # 3. Hitung cosine similarity
    print("  Menghitung cosine similarity...")
    # case_embeddings sudah berupa (n_cases, embedding_dim)
    # query_embedding sudah berupa (1, embedding_dim)
    similarities = cosine_similarity(query_embedding, case_embeddings)

    # similarities adalah array 2D [[s1, s2, ...]], ambil baris pertama
    similarities = similarities[0]

    # 4. Dapatkan top-k indices
    # argsort mengembalikan indeks yang akan mengurutkan array (dari terkecil ke terbesar)
    # jadi kita ambil dari belakang untuk skor tertinggi
    top_k_indices = np.argsort(similarities)[-k:][::-1] # Ambil k terakhir, lalu balik urutannya

    # 5. Kembalikan case_id yang sesuai
    top_k_case_ids = [case_ids_for_embeddings[i] for i in top_k_indices]
    top_k_scores = [similarities[i] for i in top_k_indices]

    print("  Top-k hasil retrieval (case_id: score):")
    for case_id, score in zip(top_k_case_ids, top_k_scores):
        print(f"    {case_id}: {score:.4f}")

    return top_k_case_ids, top_k_scores

In [ ]:
# --- 6. Persiapan dan Penyimpanan Test Queries (queries.json) ---
# [cite: 200]
# Buat beberapa contoh query dan tentukan secara manual case_id yang relevan dari data Anda
# Ini akan digunakan juga di Tahap Evaluasi.
# Anda perlu menyesuaikan query_text dan relevant_case_ids berdasarkan data Anda.

test_queries_data = [
    {
        "query_id": "Q001",
        "query_text": "Suami meninggalkan istri dan anak lebih dari dua tahun tanpa nafkah dan kabar berita.",
        "relevant_case_ids": [2, 4, 10, 12, 14, 22, 23, 29, 34, 35] # Isi dengan case_id yang relevan dari data Anda setelah inspeksi
    },
    {
        "query_id": "Q002",
        "query_text": "Terjadi perselisihan dan pertengkaran terus menerus dalam rumah tangga sehingga tidak ada harapan rukun kembali.",
        "relevant_case_ids": [1, 2, 3, 4, 7, 10, 12, 14, 21, 22, 23, 25, 29, 30, 32, 33, 34, 35] # Isi dengan case_id yang relevan
    },
    {
        "query_id": "Q003",
        "query_text": "Salah satu pihak melakukan penganiayaan.", # Ganti dengan domain yang relevan jika ini perceraian
        "relevant_case_ids": [1, 2, 7, 10, 14, 21]
    }
]

# Jika Anda sudah punya case_ids_for_embeddings, Anda bisa pilih beberapa untuk jadi ground truth
# Contoh: jika case_ids_for_embeddings = [101, 102, 103, 104, 105, ...] (ID kasus Anda)
# Maka Anda bisa set: test_queries_data[0]['relevant_case_ids'] = [102, 105] (misalnya)

print(f"\nMenyimpan contoh queries ke {QUERIES_JSON_FILE}...")
try:
    with open(QUERIES_JSON_FILE, 'w', encoding='utf-8') as f:
        json.dump(test_queries_data, f, indent=4, ensure_ascii=False)
    print("Contoh queries berhasil disimpan.")
    print(f"Silakan EDIT file '{QUERIES_JSON_FILE}' untuk mengisi 'relevant_case_ids' yang sesuai dengan data Anda.")
except Exception as e:
    print(f"Error saat menyimpan queries.json: {e}")


Menyimpan contoh queries ke /content/drive/MyDrive/Semester 6/PK/UAS/data/eval/queries.json...
Contoh queries berhasil disimpan.
Silakan EDIT file '/content/drive/MyDrive/Semester 6/PK/UAS/data/eval/queries.json' untuk mengisi 'relevant_case_ids' yang sesuai dengan data Anda.


In [ ]:
# --- 7. Pengujian Awal Fungsi Retrieval ---
print("\n--- Pengujian Awal Fungsi Retrieval ---")
if os.path.exists(QUERIES_JSON_FILE):
    try:
        with open(QUERIES_JSON_FILE, 'r', encoding='utf-8') as f:
            test_queries_for_run = json.load(f)

        # Uji dengan query pertama dari file
        if test_queries_for_run:
            sample_query = test_queries_for_run[0]
            print(f"Menguji dengan Query ID: {sample_query['query_id']}")
            retrieved_ids, retrieved_scores = retrieve_cases_bert(sample_query['query_text'], k=5)
            print(f"  ID Kasus yang berhasil di-retrieve: {retrieved_ids}")
            print(f"  Skor kemiripan: {retrieved_scores}")
            if sample_query.get('relevant_case_ids'): # Jika Anda sudah mengisi ground truth
                 print(f"  Ground truth relevant_case_ids: {sample_query['relevant_case_ids']}")
        else:
            print("File queries.json kosong atau tidak valid.")

    except Exception as e:
        print(f"Error saat memuat atau menjalankan query dari queries.json: {e}")
else:
    print(f"File {QUERIES_JSON_FILE} tidak ditemukan. Pengujian awal dilewati.")

print("\n--- Tahap 3 Selesai (Implementasi Dasar Retrieval dengan BERT) ---")
print("Output utama tahap ini adalah fungsi retrieve_cases_bert() yang teruji secara kualitatif")
print("dan file queries.json yang perlu Anda lengkapi ground truth-nya.")
print(f"Embeddings kasus disimpan di: {CASE_EMBEDDINGS_FILE}")


--- Pengujian Awal Fungsi Retrieval ---
Menguji dengan Query ID: Q001

Melakukan retrieval untuk query: "Suami meninggalkan istri dan anak lebih dari dua tahun tanpa nafkah dan kabar berita...."
  Menghasilkan embedding untuk query...
  Menghitung cosine similarity...
  Top-k hasil retrieval (case_id: score):
    30: 0.4379
    2: 0.4276
    29: 0.4036
    7: 0.3957
    23: 0.3881
  ID Kasus yang berhasil di-retrieve: [30, 2, 29, 7, 23]
  Skor kemiripan: [np.float32(0.43792742), np.float32(0.42757782), np.float32(0.40357864), np.float32(0.39570495), np.float32(0.3880915)]
  Ground truth relevant_case_ids: [2, 4, 10, 12, 14, 22, 23, 29, 34, 35]

--- Tahap 3 Selesai (Implementasi Dasar Retrieval dengan BERT) ---
Output utama tahap ini adalah fungsi retrieve_cases_bert() yang teruji secara kualitatif
dan file queries.json yang perlu Anda lengkapi ground truth-nya.
Embeddings kasus disimpan di: /content/drive/MyDrive/Semester 6/PK/UAS/data/processed/case_embeddings_bert.npy


# TF-IDF

In [ ]:
# ==============================================================================
# TF-IDF + COSINE SIMILARITY
# ==============================================================================
# Bagian ini mengimplementasikan metode retrieval statistik sebagai perbandingan.

import nltk
import re
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
import joblib # Untuk menyimpan dan memuat model/vectorizer

# --- 1. Persiapan dan Preprocessing Teks untuk TF-IDF ---
print("\n--- Memulai Pendekatan 1: TF-IDF + Cosine Similarity ---")

# Unduh daftar stopwords bahasa Indonesia dari NLTK jika belum ada
try:
    nltk.data.find('corpora/stopwords')
except LookupError:
    print("Mengunduh package 'stopwords' untuk NLTK...")
    nltk.download('stopwords', quiet=True)
    print("'stopwords' berhasil diunduh.")

# Ambil daftar stopwords bahasa Indonesia
stop_words = list(stopwords.words('indonesian'))
print(f"Jumlah stopwords bahasa Indonesia: {len(stop_words)}")

def preprocess_for_tfidf(text):
    """
    Fungsi untuk membersihkan dan melakukan preprocessing teks khusus untuk TF-IDF.
    - Lowercasing
    - Menghapus karakter non-alfanumerik
    - Menghapus stopwords
    """
    if not isinstance(text, str):
        return ""
    # Lowercasing
    text = text.lower()
    # Hapus semua karakter kecuali huruf dan spasi
    text = re.sub(r'[^a-z\s]', ' ', text)
    # Tokenisasi dan hapus stopwords
    words = text.split()
    filtered_words = [word for word in words if word not in stop_words and len(word) > 2] # Hanya ambil kata > 2 huruf
    return " ".join(filtered_words)

# Terapkan preprocessing pada kolom 'text_full' dari DataFrame Anda
print("\nMelakukan preprocessing teks untuk TF-IDF (stopword removal, dll.)...")
# df_cases sudah dimuat di awal notebook
df_cases['text_for_tfidf'] = df_cases['text_full'].apply(preprocess_for_tfidf)
print("Preprocessing teks selesai.")
print("Contoh teks setelah preprocessing:")
print(df_cases[['text_full', 'text_for_tfidf']].head())


# --- 2. Membuat dan Menyimpan Matriks TF-IDF & Vectorizer ---

# Path untuk menyimpan model TF-IDF
TFIDF_VECTORIZER_FILE = os.path.join(PATH_PROCESSED_DATA, "tfidf_vectorizer.pkl")
TFIDF_MATRIX_FILE = os.path.join(PATH_PROCESSED_DATA, "tfidf_matrix.npz")
CASE_IDS_TFIDF_FILE = os.path.join(PATH_PROCESSED_DATA, "case_ids_tfidf.json")

# Inisialisasi TfidfVectorizer
# max_df=0.85 -> abaikan kata yang muncul di lebih dari 85% dokumen
# min_df=2 -> abaikan kata yang muncul kurang dari 2 kali
tfidf_vectorizer = TfidfVectorizer(max_df=0.85, min_df=2, ngram_range=(1, 2))

print("\nMembuat matriks TF-IDF untuk case base...")
# Fit dan transform korpus
corpus_tfidf = df_cases['text_for_tfidf'].tolist()
tfidf_matrix = tfidf_vectorizer.fit_transform(corpus_tfidf)

# Simpan urutan case_id yang sesuai dengan matriks
case_ids_for_tfidf = df_cases['case_id'].tolist()

print(f"Matriks TF-IDF berhasil dibuat. Shape: {tfidf_matrix.shape}")

# Simpan vectorizer, matriks, dan urutan ID
try:
    joblib.dump(tfidf_vectorizer, TFIDF_VECTORIZER_FILE)
    print(f"Vectorizer TF-IDF berhasil disimpan ke: {TFIDF_VECTORIZER_FILE}")

    # Untuk menyimpan sparse matrix, gunakan format .npz dari scipy
    from scipy.sparse import save_npz
    save_npz(TFIDF_MATRIX_FILE, tfidf_matrix)
    print(f"Matriks TF-IDF berhasil disimpan ke: {TFIDF_MATRIX_FILE}")

    with open(CASE_IDS_TFIDF_FILE, 'w') as f:
        json.dump(case_ids_for_tfidf, f)
    print(f"Urutan Case IDs untuk TF-IDF disimpan ke: {CASE_IDS_TFIDF_FILE}")
except Exception as e:
    print(f"Error saat menyimpan model TF-IDF: {e}")


# --- 3. Implementasi Fungsi Retrieval TF-IDF ---

# Muat kembali model dan matriks yang tersimpan untuk digunakan di dalam fungsi
# Ini adalah praktik yang baik, menunjukkan bagaimana sistem akan bekerja setelah training/fitting
try:
    loaded_tfidf_vectorizer = joblib.load(TFIDF_VECTORIZER_FILE)
    from scipy.sparse import load_npz
    loaded_tfidf_matrix = load_npz(TFIDF_MATRIX_FILE)
    with open(CASE_IDS_TFIDF_FILE, 'r') as f:
        loaded_case_ids_for_tfidf = json.load(f)
    print("\nModel TF-IDF, matriks, dan ID berhasil dimuat untuk digunakan di fungsi retrieval.")
except Exception as e:
    print(f"Gagal memuat model/matriks TF-IDF yang tersimpan: {e}")
    # Gunakan variabel dari memori sebagai fallback
    loaded_tfidf_vectorizer = tfidf_vectorizer
    loaded_tfidf_matrix = tfidf_matrix
    loaded_case_ids_for_tfidf = case_ids_for_tfidf


def retrieve_cases_tfidf(query_text, k=5):
    """
    Menerima query teks, melakukan preprocessing, mengubahnya menjadi vektor TF-IDF,
    dan mengembalikan top-k case_id yang paling mirip berdasarkan cosine similarity.
    """
    if not query_text.strip():
        print("Query teks kosong.")
        return [], []

    print(f"\nMelakukan retrieval (TF-IDF) untuk query: \"{query_text[:100]}...\"")

    # 1. Preprocessing query
    processed_query = preprocess_for_tfidf(query_text)

    # 2. Transform query menjadi vektor TF-IDF menggunakan vectorizer yang sudah di-load
    query_vector = loaded_tfidf_vectorizer.transform([processed_query])

    # 3. Hitung cosine similarity
    similarities = cosine_similarity(query_vector, loaded_tfidf_matrix)[0]

    # 4. Dapatkan top-k indices
    top_k_indices = np.argsort(similarities)[-k:][::-1]

    # 5. Kembalikan case_id dan skor yang sesuai
    top_k_case_ids = [loaded_case_ids_for_tfidf[i] for i in top_k_indices]
    top_k_scores = [similarities[i] for i in top_k_indices]

    print("  Top-k hasil retrieval (TF-IDF) (case_id: score):")
    for case_id, score in zip(top_k_case_ids, top_k_scores):
        print(f"    {case_id}: {score:.4f}")

    return top_k_case_ids, top_k_scores


# --- 4. Pengujian Awal Fungsi Retrieval TF-IDF ---
print("\n--- Pengujian Awal Fungsi Retrieval TF-IDF ---")

# Muat queries.json yang sudah ada
try:
    with open(QUERIES_JSON_FILE, 'r', encoding='utf-8') as f:
        test_queries_for_run = json.load(f)

    # Uji dengan query pertama dari file
    if test_queries_for_run:
        sample_query = test_queries_for_run[0]
        print(f"Menguji dengan Query ID: {sample_query['query_id']}")
        retrieved_ids_tfidf, retrieved_scores_tfidf = retrieve_cases_tfidf(sample_query['query_text'], k=5)
        print(f"  ID Kasus yang berhasil di-retrieve (TF-IDF): {retrieved_ids_tfidf}")
        if sample_query.get('relevant_case_ids'):
             print(f"  Ground truth relevant_case_ids: {sample_query['relevant_case_ids']}")
    else:
        print("File queries.json kosong atau tidak valid.")
except FileNotFoundError:
    print(f"File {QUERIES_JSON_FILE} tidak ditemukan. Pengujian awal TF-IDF dilewati.")


--- Memulai Pendekatan 1: TF-IDF + Cosine Similarity ---
Jumlah stopwords bahasa Indonesia: 758

Melakukan preprocessing teks untuk TF-IDF (stopword removal, dll.)...
Preprocessing teks selesai.
Contoh teks setelah preprocessing:
                                           text_full  \
0  Direktori Putusan Mahkamah Agung Republik Indo...   
1  Direktori Putusan Mahkamah Agung Republik Indo...   
2  Direktori Putusan Mahkamah Agung Republik Indo...   
3  Direktori Putusan Mahkamah Agung Republik Indo...   
4  Direktori Putusan Mahkamah Agung Republik Indo...   

                                      text_for_tfidf  
0  direktori putusan mahkamah agung republik indo...  
1  direktori putusan mahkamah agung republik indo...  
2  direktori putusan mahkamah agung republik indo...  
3  direktori putusan mahkamah agung republik indo...  
4  direktori putusan mahkamah agung republik indo...  

Membuat matriks TF-IDF untuk case base...
Matriks TF-IDF berhasil dibuat. Shape: (18, 3376)
Vectorize